In [1]:
import pandas as pd
import numpy as np
from itertools import product
from scipy.optimize import minimize
from multiprocessing import Pool, cpu_count

In [2]:
actual_df = pd.read_csv('data/TradingTrackData/Actual_quantiles_train_set.csv')
dap_df = pd.read_csv('data/TradingTrackData/DAP_quantiles_train_set.csv')
ssp_df = pd.read_csv('data/TradingTrackData/SSP_quantiles_train_set.csv')
dass_df = pd.read_csv('data/TradingTrackData/DASS_train_set.csv')

In [3]:
actual_df = pd.read_csv('data/TradingTrackData/Actual_quantiles_cv_set.csv')
dap_df = pd.read_csv('data/TradingTrackData/DAP_quantiles_cv_set.csv')
ssp_df = pd.read_csv('data/TradingTrackData/SSP_quantiles_cv_set.csv')
dass_df = pd.read_csv('data/TradingTrackData/DASS_cv_set.csv')

In [4]:
actual_df = pd.read_csv('data/TradingTrackData/Actual_quantiles_test_set.csv')
dap_df = pd.read_csv('data/TradingTrackData/DAP_quantiles_test_set.csv')
ssp_df = pd.read_csv('data/TradingTrackData/SSP_quantiles_test_set.csv')
dass_df = pd.read_csv('data/TradingTrackData/DASS_test_set.csv')

In [5]:
# Merge dataframes on time to align Actual, DAP, SSP, and DASS quantiles
quantiles = ['q10', 'q20', 'q30', 'q40', 'q50', 'q60', 'q70', 'q80', 'q90']
merged_df = (
    actual_df[['time', 'total_generation_MWh'] + quantiles]
    .merge(dap_df[['time', 'DA_Price'] + quantiles], on='time', suffixes=('_actual', '_dap'))
    .merge(ssp_df[['time', 'SS_Price'] + quantiles], on='time', suffixes=('', '_ssp'))
    .merge(dass_df[['time'] + quantiles], on='time', suffixes=('', '_dass'))
)

# Rename the columns from ssp_df to include '_ssp' suffix
for quantile in quantiles:
    merged_df.rename(columns={quantile: f"{quantile}_ssp"}, inplace=True)

# Define the expected revenue function
def revenue(bid, y, day_ahead_price, single_system_price):
    return bid * day_ahead_price + (y - bid) * (single_system_price - 0.07 * (y - bid))

def expected_revenue(trade, quantile_probs, actual_quantiles, dap_quantiles, ssp_quantiles):
    revenue_sum = 0
    scenario_count = 0

    # Use itertools.product to generate all combinations of actual, dap, and ssp quantiles
    for actual, dap, ssp in product(actual_quantiles, dap_quantiles, ssp_quantiles):
        scenario_count += 1
        
        # Calculate revenue for the current scenario
        revenue_sum += quantile_probs[0] * (
            trade * dap + (actual - trade) * (ssp - 0.07 * (actual - trade))
        )
        
    return revenue_sum

# Function to be minimized (negative expected revenue for minimization)
def neg_expected_revenue(trade, quantile_probs, actual_quantiles, dap_quantiles, ssp_quantiles):
    return -expected_revenue(trade, quantile_probs, actual_quantiles, dap_quantiles, ssp_quantiles)

# Probabilities associated with each quantile (assuming uniform distribution)
quantile_probs = [1/9] * len(quantiles)

# Constraints for the optimization
bounds = [(0, 1800)]  # Trade between 0 and 1800

# Function to optimize trade for a single row (to be parallelized)
def optimize_trade(row):
    actual_quantiles = [row[q + '_actual'] for q in quantiles]
    dap_quantiles = [row[q + '_dap'] for q in quantiles]
    ssp_quantiles = [row[q + '_ssp'] for q in quantiles]
    
    # Initial guess for trade is set to Actual_q50
    initial_trade = row['q50_actual']
    
    # Perform the optimization
    result = minimize(
        neg_expected_revenue, 
        initial_trade, 
        args=(quantile_probs, actual_quantiles, dap_quantiles, ssp_quantiles), 
        bounds=bounds, 
        method='L-BFGS-B'
    )
    
    # Extract the optimized trade
    return result.x[0]

# Use multiprocessing to optimize trades for all rows
with Pool(cpu_count()) as pool:
    optimized_trades = pool.map(optimize_trade, [row for _, row in merged_df.iterrows()])

# Add the optimized trade to the dataframe
merged_df['optimized_trade'] = optimized_trades

import random

# Function to choose a random quantile for each row
def choose_random_quantile(row):
    quantiles = ['q10_actual', 'q20_actual', 'q30_actual', 'q40_actual', 'q50_actual', 'q60_actual', 'q70_actual', 'q80_actual', 'q90_actual']
    return row[random.choice(quantiles)]

# Apply the function to each row to generate the random strategy trade
merged_df['random_trade'] = merged_df.apply(choose_random_quantile, axis=1)

print("Optimization complete. Results saved to 'optimized_trade.csv'.")

Optimization complete. Results saved to 'optimized_trade.csv'.


In [6]:
def revenue(bid,y,day_ahead_price,single_system_price):
    return bid*day_ahead_price + (y-bid)*(single_system_price - 0.07*(y-bid))

# Calculate revenue for the random strategy
random_strategy_revenue = revenue(
    bid=merged_df['random_trade'],
    y=merged_df['total_generation_MWh'],
    day_ahead_price=merged_df['DA_Price'],
    single_system_price=merged_df['SS_Price']
).sum()

# Competition Benchmark Revenue (Bid always q50)
q10 = revenue(bid=merged_df['q10_actual'],
                            y=merged_df['total_generation_MWh'],
                            day_ahead_price=merged_df['DA_Price'],
                            single_system_price=merged_df['SS_Price']).sum()

# Competition Benchmark Revenue (Bid always q50)
q20 = revenue(bid=merged_df['q20_actual'],
                            y=merged_df['total_generation_MWh'],
                            day_ahead_price=merged_df['DA_Price'],
                            single_system_price=merged_df['SS_Price']).sum()

# Competition Benchmark Revenue (Bid always q50)
q30 = revenue(bid=merged_df['q30_actual'],
                            y=merged_df['total_generation_MWh'],
                            day_ahead_price=merged_df['DA_Price'],
                            single_system_price=merged_df['SS_Price']).sum()

# Competition Benchmark Revenue (Bid always q50)
q40 = revenue(bid=merged_df['q40_actual'],
                            y=merged_df['total_generation_MWh'],
                            day_ahead_price=merged_df['DA_Price'],
                            single_system_price=merged_df['SS_Price']).sum()

# Competition Benchmark Revenue (Bid always q50)
q60 = revenue(bid=merged_df['q60_actual'],
                            y=merged_df['total_generation_MWh'],
                            day_ahead_price=merged_df['DA_Price'],
                            single_system_price=merged_df['SS_Price']).sum()

# Competition Benchmark Revenue (Bid always q50)
q70 = revenue(bid=merged_df['q70_actual'],
                            y=merged_df['total_generation_MWh'],
                            day_ahead_price=merged_df['DA_Price'],
                            single_system_price=merged_df['SS_Price']).sum()

# Competition Benchmark Revenue (Bid always q50)
q80 = revenue(bid=merged_df['q80_actual'],
                            y=merged_df['total_generation_MWh'],
                            day_ahead_price=merged_df['DA_Price'],
                            single_system_price=merged_df['SS_Price']).sum()

# Competition Benchmark Revenue (Bid always q50)
q90 = revenue(bid=merged_df['q90_actual'],
                            y=merged_df['total_generation_MWh'],
                            day_ahead_price=merged_df['DA_Price'],
                            single_system_price=merged_df['SS_Price']).sum()

# Competition Benchmark Revenue (Bid always q50)
benchmark = revenue(bid=merged_df['q50_actual'],
                            y=merged_df['total_generation_MWh'],
                            day_ahead_price=merged_df['DA_Price'],
                            single_system_price=merged_df['SS_Price']).sum()

# Competition Benchmark Revenue (Bid always q50)
benchmark_theoretical = revenue(bid=merged_df['total_generation_MWh'],
                            y=merged_df['total_generation_MWh'],
                            day_ahead_price=merged_df['DA_Price'],
                            single_system_price=merged_df['SS_Price']).sum()

# Competition Benchmark Revenue (Bid always q50)
theoretical_optimal = revenue(bid=np.clip(merged_df['total_generation_MWh'] + 7.14*(merged_df['DA_Price'] - merged_df['SS_Price']), 0, 1800),
                            y=merged_df['total_generation_MWh'],
                            day_ahead_price=merged_df['DA_Price'],
                            single_system_price=merged_df['SS_Price']).sum()

# Competition Benchmark Revenue (Bid always q50)
predicted_optimal = revenue(bid=np.clip(merged_df['q50_actual'] + 7.14*(merged_df['q50_dap'] - merged_df['q50_ssp']), 0, 1800),
                            y=merged_df['total_generation_MWh'],
                            day_ahead_price=merged_df['DA_Price'],
                            single_system_price=merged_df['SS_Price']).sum()

# Competition Benchmark Revenue (Bid always q50)
predicted_optimal_dass = revenue(bid=np.clip(merged_df['q50_actual'] + 7.14*(merged_df['q50_dass']), 0, 1800),
                            y=merged_df['total_generation_MWh'],
                            day_ahead_price=merged_df['DA_Price'],
                            single_system_price=merged_df['SS_Price']).sum()

# Competition Benchmark Revenue (Bid always q50)
stochastic_optimal = revenue(bid=merged_df['optimized_trade']  ,
                            y=merged_df['total_generation_MWh'],
                            day_ahead_price=merged_df['DA_Price'],
                            single_system_price=merged_df['SS_Price']).sum()

# Calculate percentages relative to the theoretical optimal
q10_percentage = (q10 / theoretical_optimal) * 100
q20_percentage = (q20 / theoretical_optimal) * 100
q30_percentage = (q30 / theoretical_optimal) * 100
q40_percentage = (q40 / theoretical_optimal) * 100
benchmark_percentage = (benchmark / theoretical_optimal) * 100
q60_percentage = (q60 / theoretical_optimal) * 100
q70_percentage = (q70 / theoretical_optimal) * 100
q80_percentage = (q80 / theoretical_optimal) * 100
q90_percentage = (q90 / theoretical_optimal) * 100
random_strategy_percentage = (random_strategy_revenue / theoretical_optimal) * 100
benchmark_theoretical_percentage = (benchmark_theoretical / theoretical_optimal) * 100
predicted_optimal_percentage = (predicted_optimal / theoretical_optimal) * 100
predicted_optimal_dass_percentage = (predicted_optimal_dass / theoretical_optimal) * 100
stochastic_optimal_percentage = (stochastic_optimal / theoretical_optimal) * 100

In [7]:
import pandas as pd

# Data for the table
data = {
    'Type of Strategy': [
        'Q10',
        'Q20',
        'Q30',
        'Q40',
        'Benchmark',
        'Q60',
        'Q70',
        'Q80',
        'Q90', 
        'Random',
        'MAX (Theoretical Optimal)', 
        'Optimal' ,
        'Optimal based on DASS',
        'Stochastic Optimal'
    ],
    'Revenue (£ mil)': [
        q10 / 1e6,
        q20 / 1e6,
        q30 / 1e6,
        q40 / 1e6,
        benchmark / 1e6,
        q60 / 1e6,
        q70 / 1e6,
        q80 / 1e6,
        q90 / 1e6,
        random_strategy_revenue / 1e6, 
        theoretical_optimal / 1e6, 
        predicted_optimal / 1e6,
        predicted_optimal_dass / 1e6, 
        stochastic_optimal / 1e6
    ],
    '(%)': [
        q10_percentage,
        q20_percentage,
        q30_percentage,
        q40_percentage,
        benchmark_percentage,
        q60_percentage,
        q70_percentage,
        q80_percentage,
        q90_percentage, 
        random_strategy_percentage,
        100.00,  # Theoretical Optimal is 100%
        predicted_optimal_percentage,
        predicted_optimal_dass_percentage, 
        stochastic_optimal_percentage
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)
# Format the Revenue column to show values in millions with two decimal places
df['Revenue (£ mil)'] = df['Revenue (£ mil)'].apply(lambda x: f"{x:.2f}")
# Print the DataFrame
print(df)


             Type of Strategy Revenue (£ mil)         (%)
0                         Q10           86.91   83.153227
1                         Q20           87.87   84.078473
2                         Q30           88.46   84.634757
3                         Q40           88.62   84.792784
4                   Benchmark           88.57   84.741835
5                         Q60           88.36   84.542369
6                         Q70           87.90   84.100475
7                         Q80           86.75   83.002498
8                         Q90           84.08   80.448162
9                      Random           87.37   83.596093
10  MAX (Theoretical Optimal)          104.51  100.000000
11                    Optimal           88.44   84.622330
12      Optimal based on DASS           88.01   84.205827
13         Stochastic Optimal           88.91   85.073657
